In [64]:
def find(s, ch):
    return [i for i, ltr in enumerate(s) if ltr == ch]

def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub)

class Node:
    question_number = -1
    follow_up_number = 0
    raw_question = ""
    shortened_question = ""
    references = []
    instructor_answer = ""
    student_answer = ""
    question_folders = []
    
    
    def __init__(self, question_number, follow_up_number, raw_question, shortened_question, instructor_answer, student_answer, question_folders):
        self.question_number = question_number
        self.raw_question = raw_question
        self.follow_up_number = follow_up_number
        self.raw_question = raw_question
        self.instructor_answer = instructor_answer
        self.student_answer = student_answer
        self.shortened_question = shortened_question
        self.question_folders = question_folders
        
    def __str__(self):
        return self.question_number
    
    def add_references(self, all_refs):
        self.references = all_refs

In [2]:
import nltk
from nltk.tokenize import word_tokenize
import string 

def shortenQuestions(stop_words, question):

    shortened_questions = []

    tokens = word_tokenize(question)
    tokens = [w.lower() for w in tokens]
    words = [w for w in tokens if (not w in stop_words and w != "" and w != " ")]

    # Get rid of punctuation
    table = str.maketrans('', '', string.punctuation)
    words = [w.translate(table) for w in words]

    final_question = " ".join(words)
    return final_question

def createStopWords(questions):
    
    questionsText = " ".join(questions)
    
    stop_words = ["ourselves", "hers", "between", "yourself", "but", "again", 
                      "there", "about", "once", "during", "out", "very", "having", 
                      "with", "they", "own", "an", "be", "some", "for", "do", "its", 
                      "yours", "such", "into", "of", "most", "itself", "other", "off",
                      "is", "s", "am", "or", "who", "as", "from", "him", "each", "the",
                      "themselves", "until", "below", "are", "we", "these", "your", "his", 
                      "through", "don", "nor", "me", "were", "her", "more", "himself", "this",
                      "down", "should", "our", "their", "while", "above", "both", "up", "to",
                      "ours", "had", "she", "all", "no", "when", "at", "any", "before", "them",
                      "same", "and", "been", "have", "in", "will", "on", "does", "yourselves", 
                      "then", "that", "because", "what", "over", "why", "so", "can", "did", 
                      "not", "now", "under", "he", "you", "herself", "has", "just",
                      "where", "too", "only", "myself", "which", "those", "i", "after",
                      "few", "whom", "t", "being", "if", "theirs", "my", "against", "a",
                      "by", "doing", "it", "how", "further", "was", "here", "than", "ok",
                      "thank", "ah"]
        
    entireTextTokens = nltk.tokenize.word_tokenize(questionsText)
    
    # Get rid of punctuation
    table = str.maketrans('', '', string.punctuation)
    entireTextTokens = [w.translate(table) for w in entireTextTokens]

    entireTextDist = nltk.FreqDist(w.lower() for w in entireTextTokens if w and w not in stop_words)

    mostCommon = entireTextDist.most_common(30)

    for i in mostCommon:
        stop_words.append(i[0])
        print(i[0])
        
    return stop_words

In [29]:
import csv
import datetime

filename = 'contributions.csv'
all_contributions = {}
raw_references = {}
raw_timestamps_to_hw = {}
all_folders = []

# reading csv file 
with open(filename, 'r', encoding="utf8") as csvfile: 
    
    # creating a csv reader object 
    csvreader = csv.reader(csvfile) 
      
    # extracting field names through first row 
    fields = next(csvreader) 
    
    rows = []
    entireText = ""
    
    # extracting each data row one by one 
    for row in csvreader: 
        rows.append(row) 
        entireText += " ".join(row)
        
    # stop_words = createStopWords(entireText)

KeyboardInterrupt: 

In [65]:
curr_text = ""
curr_post_number = ""
curr_folder = ""
folders = []
all_folders = []
curr_timestamp = ""
final_column_question = ""
final_column_instructor_answer = ""
final_column_student_answer = ""

for row in rows[:]: 
    # parsing each column of a row 
    for col,field in zip(row, fields): 
        if(field == "Post Number"):
            if not curr_post_number == "" :

                if curr_timestamp[-3:] == 'UTC':
                    curr_timestamp = curr_timestamp[:-4]
                timestamp = datetime.datetime.strptime(curr_timestamp, '%Y-%m-%d %H:%M:%S')

                #shortened_question = shortenQuestions(stop_words, final_column_question)
                shortened_question= ""
                node = Node(curr_post_number, 0, final_column_question, shortened_question, final_column_instructor_answer, final_column_student_answer, folders)
                all_contributions[(curr_post_number, 0)] = node
                for folder in curr_folders:
                    hw_timestamps = []
                    if folder in raw_timestamps_to_hw:
                        hw_timestamps = raw_timestamps_to_hw[folder]

                    hw_timestamps.append(timestamp)
                    raw_timestamps_to_hw.update({folder: hw_timestamps})

                for loc in find_all(final_column_instructor_answer, "@"):
                    next_space = loc + 1
                    while(next_space <= len(final_column_instructor_answer)-1 and final_column_instructor_answer[next_space].isdigit()):
                        next_space = next_space + 1

                    question = -1
                    try:
                        question = int(final_column_instructor_answer[(loc + 1):next_space])
                    except ValueError:
                        question = -1

                    raw_references_q = []
                    if curr_post_number in raw_references:
                        raw_references_q = raw_references[curr_post_number]

                    raw_references_q.append(question)
                    raw_references.update({curr_post_number : raw_references_q})

                    raw_references_q = []
                    if question in raw_references:
                        raw_references_q = raw_references[question]

                    raw_references_q.append(curr_post_number)
                    raw_references.update({question : raw_references_q})

            curr_post_number = col

        elif(field == "Submission HTML Removed"): #the question could also be in subject
            curr_text = col

        elif(field == "Folders"):
            curr_folder = col
            
            folders = []
            curr_folders = curr_folder.split()
            for folder in curr_folders:
                new_folder = folder
                if folder[-1] == ';':
                    new_folder = folder[:-1]
                folders.append(new_folder)
                
                if new_folder not in all_folders:
                    all_folders.append(new_folder) 

        elif(field == "Created At"):
            curr_timestamp = col

        elif(field == "Part of Post"):
            if(col == "started_off_question"):
                final_column_question = curr_text

            elif(col == "updated_question"):
                # must sort the csv by question number and then by time (so the updated question is posted after the og)
                final_column_question = curr_text

            elif(col == "updated_i_answer"):
                final_column_instructor_answer = curr_text

            elif(col == "started_off_i_answer"):
                final_column_instructor_answer = curr_text

            elif(col == "updated_s_answer"):
                final_column_student_answer = curr_text

            elif(col == "started_off_s_answer"):
                final_column_student_answer = curr_text

            elif(col == "followup"):
                continue

            elif(col == "reply_to_followup"):
                continue

for key in raw_references:
    if not key == -1:
        all_contributions[(str(key),0)].add_references(raw_references[key])
        
# for folder in all_folders:
#     print(folder)

# for key in all_contributions:
#     print(key)
#     contribution = all_contributions[key]
#     print("Question Number: %s \n Question: %s \n Shortened Question: %s \n Answer: %s \n %s \n %s \n" % \
#           (contribution.question_number, contribution.raw_question, contribution.shortened_question, contribution.instructor_answer, contribution.references, contribution.question_folders))

logistics
other
exam
hw1
hw2
hw3
hw4
hw5
hw6
hw7
hw8
hw9
hw10
project
polls
('6', 0)
Question Number: 6 
 Question:  
 Shortened Question:  
 Answer:  
 [] 
 ['logistics'] 

('7', 0)
Question Number: 7 
 Question: Hi! I was registered half an hour ago for both the lecture and recitation and just tried to switch my recitation to better fit my schedule. I dropped my old recitation and when I tried to add a new one the, the system wouldn't let me. I would like to switch into 214, 216, 217, all of which Penn in Touch says are open. I would, at this point, also be happy to take my old spot back, which was in 211, which I believe is now closed. I am very concerned that without the recitation slot, Penn in Touch will soon delete my lecture spot, as well. Could someone please help me re-register for recitation? 

Thank you!
Shira Silver
Penn ID: 76417234 
 Shortened Question:  
 Answer: You should fill out the following form (also found on the home page of the CIS 120 website):

https://docs.g

Question Number: 691 
 Question: Are we allowed to use the @ sign in Q4 inorder? 
 Shortened Question:  
 Answer: Yes 
 [] 
 ['hw3'] 

('692', 0)
Question Number: 692 
 Question: I am trying to test my listSet, but for some reason it is testing treeList first which I have not yet completed yet so it won't let me see if I am doing any of listSet correctly. How can I fix this? 
 Shortened Question:  
 Answer: @656 
 [656, 656, 656, 656, 656, 656, 656, 656, 656, 656, 656, 656, 656, 656, 656, 656, 656, 656, 656, 656, 656, 656, 656, 656, 656, 656] 
 ['hw3'] 

('693', 0)
Question Number: 693 
 Question: do we get any late days without getting 10% off? or is our first late submission 10% off 
 Shortened Question:  
 Answer: No, the syllabus is here. 
 [656, 656, 656, 656, 656, 656, 656, 656, 656, 656, 656, 656, 656, 656, 656, 656, 656, 656, 656, 656, 656, 656, 656, 656, 656, 656] 
 ['logistics'] 

('694', 0)
Question Number: 694 
 Question: Hi,
Can I say if size s = size t and then inorder s 


('1334', 0)
Question Number: 1334 
 Question: Is there any statistics on the mean for the past midterms posted on the website? 
 Shortened Question:  
 Answer: I don't believe so 
 [] 
 ['exam'] 

('1335', 0)
Question Number: 1335 
 Question: Why is this ill typed?
let e : ________________________________ = begin match (remove 1 empty) with
                                                                                | [] -&gt; true
                                                                                | _ -&gt; false end 
 Shortened Question:  
 Answer: What is the type of remove 1 empty? Is it a list?  
 [] 
 ['exam'] 

('1336', 0)
Question Number: 1336 
 Question: Would this implementation of the function work? Assuming the max helper function is kept the same. I tried running it on codio but I couldn't find a way to run the tests.

begin match l with | [] -&gt; "" | hd :: tl -&gt; if String.length (max hd) &gt; String.length (longest tl) then max hd else longest tl end 

In [52]:
import csv

hw_timestamps_filename = 'hw_timestamps.csv'
with open(hw_timestamps_filename, 'w', encoding="utf8") as hw_timestamps_file: 
    # creating a csv reader object 
    results_writer = csv.writer(hw_timestamps_file) 

    for key, value in raw_timestamps_to_hw.items(): 
        
        print(key)
        dates = ""
        for l in value:
            dates = dates + l.strftime('%m/%d/%Y') + ","
            print(l.strftime('%m/%d/%Y'))
        results_writer.writerow([key, dates])

logistics
09/30/2018
08/29/2018
08/29/2018
08/30/2018
08/30/2018
09/05/2018
08/31/2018
09/18/2018
08/31/2018
09/01/2018
09/01/2018
09/04/2018
09/04/2018
09/05/2018
09/05/2018
09/05/2018
09/05/2018
09/05/2018
09/05/2018
09/11/2018
09/11/2018
09/12/2018
09/12/2018
09/12/2018
09/12/2018
09/14/2018
09/14/2018
09/24/2018
09/25/2018
09/19/2018
09/15/2018
09/15/2018
09/15/2018
09/15/2018
09/16/2018
09/16/2018
09/17/2018
09/17/2018
09/18/2018
09/18/2018
09/19/2018
09/19/2018
09/19/2018
09/19/2018
09/19/2018
09/19/2018
09/21/2018
09/21/2018
09/25/2018
09/21/2018
09/21/2018
09/22/2018
09/22/2018
09/22/2018
09/22/2018
09/25/2018
09/25/2018
10/14/2018
10/14/2018
09/26/2018
09/26/2018
09/26/2018
09/26/2018
09/30/2018
09/28/2018
09/28/2018
10/05/2018
09/28/2018
09/28/2018
10/03/2018
10/03/2018
10/03/2018
10/11/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/09/2018
10/09/2018
10/10/2018
10/10/2018
10/11/2018
10/10/2018
10/11/2018
10/11/2018
09/30/2018
08/29/2018
08/29/2018
08/30/2018
08/30/2018


10/12/2018
10/12/2018
09/26/2018
09/26/2018
09/26/2018
09/26/2018
09/30/2018
09/28/2018
09/28/2018
10/05/2018
09/28/2018
09/28/2018
10/03/2018
10/03/2018
10/03/2018
10/11/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/09/2018
10/09/2018
10/10/2018
10/10/2018
10/11/2018
10/10/2018
10/19/2018
10/11/2018
10/19/2018
10/19/2018
10/11/2018
10/11/2018
10/11/2018
10/11/2018
09/30/2018
08/29/2018
08/29/2018
08/30/2018
08/30/2018
09/05/2018
08/31/2018
09/18/2018
08/31/2018
09/01/2018
09/01/2018
09/04/2018
09/04/2018
09/05/2018
09/05/2018
09/05/2018
09/05/2018
09/05/2018
09/05/2018
09/11/2018
09/11/2018
09/12/2018
09/12/2018
09/12/2018
09/12/2018
09/21/2018
09/12/2018
09/14/2018
09/14/2018
09/24/2018
09/25/2018
09/19/2018
09/15/2018
09/15/2018
09/15/2018
09/15/2018
09/16/2018
09/16/2018
09/17/2018
09/17/2018
12/22/2018
10/19/2018
10/19/2018
09/17/2018
09/18/2018
09/18/2018
12/02/2018
11/21/2018
11/21/2018
11/27/2018
11/27/2018
11/27/2018
11/27/2018
11/27/2018
12/02/2018
12/11/2018
12/11/2018

10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
08/30/2018
08/30/2018
08/30/2018
08/30/2018
08/30/2018
08/30/2018
09/09/2018
08/31/2018
08/31/2018
09/07/2018
08/31/2018
08/31/2018
08/31/2018
08/31/2018
08/31/2018
08/31/2018
08/31/2018
08/31/2018
08/31/2018
09/01/2018
09/03/2018
09/05/2018
09/05/2018
09/05/2018
09/05/2018
09/05/2018
09/05/2018
09/11/2018
09/05/2018
09/05/2018
09/05/2018
09/05/2018
09/09/2018
09/07/2018
09/07/2018
09/07/2018
09/07/2018
09/12/2018
09/07/2018
09/08/2018
09/08/2018
09/08/2018
09/08/2018
09/09/2018
09/09/2018
09/09/2018
09/09/2018
09/09/2018
09/09/2018
09/15/2018
09/09/2018
09/15/2018
09/11/2018
09/11/2018
09/11/2018
09/11/2018
09/13/2018
09/13/2018
09/17/2018

10/08/2018
10/08/2018
10/09/2018
10/10/2018
10/09/2018
10/09/2018
10/09/2018
10/09/2018
10/09/2018
10/11/2018
10/11/2018
10/09/2018
10/09/2018
10/09/2018
10/09/2018
10/10/2018
10/10/2018
10/10/2018
10/10/2018
10/10/2018
10/10/2018
10/10/2018
10/10/2018
10/10/2018
10/10/2018
10/10/2018
10/10/2018
10/10/2018
10/10/2018
10/10/2018
10/10/2018
10/10/2018
10/10/2018
10/10/2018
10/11/2018
10/11/2018
10/11/2018
10/11/2018
10/10/2018
10/10/2018
10/11/2018
10/11/2018
10/11/2018
10/11/2018
10/11/2018
10/11/2018
10/11/2018
10/14/2018
10/14/2018
10/11/2018
10/11/2018
10/11/2018
10/12/2018
10/12/2018
10/12/2018
10/12/2018
10/12/2018
10/12/2018
10/12/2018
10/12/2018
10/12/2018
10/12/2018
10/12/2018
10/12/2018
10/12/2018
10/13/2018
10/14/2018
10/13/2018
10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018

10/10/2018
10/10/2018
10/10/2018
10/10/2018
10/10/2018
10/11/2018
10/11/2018
10/11/2018
10/11/2018
10/10/2018
10/10/2018
10/11/2018
10/11/2018
10/11/2018
10/11/2018
10/11/2018
10/11/2018
10/11/2018
10/14/2018
10/14/2018
10/11/2018
10/11/2018
10/11/2018
10/12/2018
10/12/2018
10/12/2018
10/12/2018
10/12/2018
10/12/2018
10/12/2018
10/12/2018
10/12/2018
10/12/2018
10/12/2018
10/12/2018
10/12/2018
10/15/2018
10/12/2018
10/12/2018
10/12/2018
10/13/2018
10/14/2018
10/13/2018
10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
08/30/2018
08/30/2018
08/30/2018
08/30/2018
08/30/2018
08/30/2018
09/09/2018
08/31/2018
08/31/2018

...

10/07/2018
10/07/2018
10/07/2018
10/07/2018
10/07/2018
10/07/2018
10/07/2018
10/07/2018
10/07/2018
10/07/2018
10/07/2018
10/07/2018
10/07/2018
10/07/2018
10/07/2018
10/07/2018
10/07/2018
10/08/2018
10/08/2018
10/07/2018
10/07/2018
10/07/2018
10/07/2018
10/07/2018
10/07/2018
10/07/2018
10/07/2018
10/07/2018
10/07/2018
10/07/2018
10/07/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/09/2018
10/09/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018
10/08/2018

10/16/2018
10/16/2018
10/17/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/19/2018
10/19/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/17/2018
10/17/2018
10/18/2018
10/18/2018
10/22/2018
10/22/2018
10/22/2018
10/16/2018
10/17/2018
10/17/2018
10/17/2018
10/17/2018
10/17/2018
10/17/2018
10/17/2018
10/17/2018
10/17/2018
10/17/2018
10/17/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/18/2018
10/18/2018
10/17/2018
10/17/2018
10/17/2018
10/17/2018
10/17/2018
10/12/2018
10/12/2018
10/24/2018
10/12/2018
10/24/2018
10/18/2018
10/18/2018
10/14/2018
10/14/2018
10/12/2018
10/12/2018
10/12/2018
10/12/2018
10/12/2018
10/13/2018
10/13/2018
10/13/2018
10/13/2018
10/13/2018
10/13/2018
10/13/2018
10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/16/2018
10/16/2018
10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/20/2018
10/20/2018
10/20/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018

10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/16/2018
10/16/2018
10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/20/2018
10/20/2018
10/20/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/17/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/19/2018
10/19/2018
10/19/2018
10/16/2018
10/16/2018
10/16/2018
10/17/2018
10/18/2018
10/18/2018
10/18/2018
10/18/2018
10/18/2018
10/20/2018
10/20/2018
10/20/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/21/2018
10/21/2018
10/21/2018
10/21/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018

10/16/2018
10/16/2018
10/17/2018
10/17/2018
10/18/2018
10/18/2018
10/22/2018
10/22/2018
10/22/2018
10/16/2018
10/17/2018
10/17/2018
10/17/2018
10/17/2018
10/17/2018
10/17/2018
10/17/2018
10/17/2018
10/17/2018
10/17/2018
10/17/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/18/2018
10/18/2018
10/17/2018
10/17/2018
10/17/2018
10/17/2018
10/17/2018
10/12/2018
10/12/2018
10/24/2018
10/12/2018
10/24/2018
10/18/2018
10/18/2018
10/14/2018
10/14/2018
10/12/2018
10/12/2018
10/12/2018
10/12/2018
10/12/2018
10/13/2018
10/13/2018
10/13/2018
10/13/2018
10/13/2018
10/13/2018
10/13/2018
10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/16/2018
10/16/2018
10/14/2018
10/14/2018
10/14/2018
10/14/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/20/2018
10/20/2018
10/20/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/15/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/16/2018
10/17/2018
10/16/2018
10/16/2018

09/24/2018
10/24/2018
09/25/2018
10/12/2018
10/12/2018
10/01/2018
10/05/2018
10/05/2018
10/05/2018
10/05/2018
10/05/2018
10/05/2018
10/05/2018
10/05/2018
10/05/2018
10/05/2018
10/06/2018
10/06/2018
10/09/2018
10/09/2018
10/09/2018
09/24/2018
09/24/2018
09/24/2018
09/24/2018
10/24/2018
09/25/2018
10/12/2018
10/12/2018
10/01/2018
10/05/2018
10/05/2018
10/05/2018
10/05/2018
10/05/2018
10/05/2018
10/05/2018
10/05/2018
10/05/2018
10/05/2018
10/06/2018
10/06/2018
10/09/2018
10/09/2018
10/09/2018
09/24/2018
09/24/2018
09/24/2018
09/24/2018
10/24/2018
09/25/2018
10/12/2018
10/12/2018
10/01/2018
10/05/2018
10/05/2018
10/05/2018
10/05/2018
10/05/2018
10/05/2018
10/05/2018
10/05/2018
10/05/2018
10/05/2018
10/06/2018
10/06/2018
10/09/2018
10/09/2018
10/09/2018
09/24/2018
09/24/2018
09/24/2018
09/24/2018
10/24/2018
09/25/2018
10/12/2018
10/12/2018
10/01/2018
10/05/2018
10/05/2018
10/05/2018
10/05/2018
10/05/2018
10/05/2018
10/05/2018
10/05/2018
10/05/2018
10/05/2018
10/06/2018
10/06/2018
10/09/2018

In [67]:
import operator
import csv

def Remove(duplicate): 
    final_list = [] 
    for num in duplicate: 
        if num not in final_list: 
            final_list.append(num) 
    return final_list 

# for key, value in raw_references.items():
#     without_duplicates = Remove(value)
#     raw_references.update({key: without_duplicates})
   
# raw_references_sorted = sorted(raw_references.items(), key=lambda v: len(v[1]))

references_filename = 'references.csv'
with open(references_filename, 'w', encoding="utf8") as references_file: 
    reference_writer = csv.writer(references_file) 
    
    for folder in all_folders:
        print(folder)
        reference_writer.writerow([folder])
        for node in all_contributions.values():
            if folder in node.question_folders:
                print({node.question_number: node.references})
                reference_writer.writerow([node.question_number, node.references])
        

logistics
{'6': []}
{'8': []}
{'16': ['146', '316', '317', '146', '316', '317', '146', '316', '317', '146', '316', '317', '146', '316', '317', '146', '316', '317', '146', '316', '317', '146', '316', '317', '146', '316', '317', '146', '316', '317', '146', '316', '317', '146', '316', '317', '146', '316', '317', '146', '316', '317', '146', '316', '317', '146', '316', '317', '146', '316', '317', '146', '316', '317', '146', '316', '317', '146', '316', '317', '146', '316', '317', '146', '316', '317', '146', '316', '317', '146', '316', '317', '146', '316', '317', '146', '316', '317']}
{'17': []}
{'18': ['68', '69', '69', '69', '70', '473', '474', '694', '695', '695', '695', '695', '695', '696', '1080', '1081', '1081', '1082', '1211', '1211', '1211', '1211', '1212', '1227', '1227', '1228', '68', '69', '69', '69', '70', '473', '474', '694', '695', '695', '695', '695', '695', '696', '1080', '1081', '1081', '1082', '1211', '1211', '1211', '1211', '1212', '1227', '1227', '1228', '68', '69', '69', 

{'997': [855, 978, 855, 978, 855, 978, 855, 978, 855, 978, 855, 978, 855, 978, 855, 978, 855, 978, 855, 978, 855, 978, 855, 978, 855, 978, 855, 978, 855, 978, 855, 978, 855, 978, 855, 978, 855, 978, 855, 978, 855, 978, 855, 978, 855, 978, 855, 978, 855, 978, 855, 978]}
{'998': [978, 996, 978, 996, 978, 996, 978, 996, 978, 996, 978, 996, 978, 996, 978, 996, 978, 996, 978, 996, 978, 996, 978, 996, 978, 996, 978, 996, 978, 996, 978, 996, 978, 996, 978, 996, 978, 996, 978, 996, 978, 996, 978, 996, 978, 996, 978, 996, 978, 996, 978, 996]}
{'999': [996, 996, 996, 996, 996, 996, 996, 996, 996, 996, 996, 996, 996, 996, 996, 996, 996, 996, 996, 996, 996, 996, 996, 996, 996, 996]}
{'1001': []}
{'1002': []}
{'1003': [963, 963, 963, 963, 963, 963, 963, 963, 963, 963, 963, 963, 963, 963, 963, 963, 963, 963, 963, 963, 963, 963, 963, 963, 963, 963]}
{'1004': [963, 963, 963, 963, 963, 963, 963, 963, 963, 963, 963, 963, 963, 963, 963, 963, 963, 963, 963, 963, 963, 963, 963, 963, 963, 963]}
{'1005': [95